In [7]:
from glob import glob
import uproot as ur
import numpy as np
import json
from phc import module_reload
module_reload('zhh')
from zhh import parse_sample_path
version = 'v1'
REPO_ROOT = '/afs/desy.de/user/b/bliewert/public/MarlinWorkdirs/ZHH'
DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh/Preselection/{version}'

In [17]:
pol = {
    "+":  0.3,
    "-": -0.8
}

# https://doi.org/10.1016/j.physrep.2007.12.003
def w_prefacs(Pem, Pep):
    return (
        (1-Pem)*(1-Pep)/4, # LL
        (1-Pem)*(1+Pep)/4, # LR
        (1+Pem)*(1-Pep)/4, # RL
        (1+Pem)*(1+Pep)/4 # RR
    )
    

w_em_ep = {
    'LL': 0.315,
    'LR': 0.585,
    'RL': 0.035,
    'RR': 0.065
}

def get_pol_key(pol_em:int, pol_ep:int)->str:
    key_em = ('L' if pol_em == -1 else ('R' if pol_em == 1 else 'N'))
    key_ep = ('L' if pol_ep == -1 else ('R' if pol_ep == 1 else 'N'))
    return key_em + key_ep

def get_w_pol(pol_em:int, pol_ep:int)->float:
    key = get_pol_key(pol_em, pol_ep)
    
    if not (key in w_em_ep):
        raise Exception(f'Unhandled polarization {key}')
    
    return w_em_ep[key]

def weighting(process_sigma_fb:float, pol:tuple[int], n_gen:int, lum_inv_ab:float=4., ):
    w_pol = get_w_pol(*pol)
    return process_sigma_fb*1000 *lum_inv_ab*w_pol/n_gen

In [18]:
SRC_NAME = '_Source.txt'
FS_META = '_FinalStateMeta.json'
finished = glob(f'{DATA_ROOT}/*{SRC_NAME}')
finished.sort()

for f in finished:
    branch = f.split(f'{version}/')[1].split(SRC_NAME)[0]
    
    with open(f, 'r') as file:
        src_file = file.read()
        
    # Read metadata
    with open(f'{DATA_ROOT}/{branch}{FS_META}', 'r') as file:
        meta = json.load(file)
        
        # {'crossSection': 133070.796875, 'crossSectionError': 78.4000015258789, 'eventWeight': 1.0, 'nEvtSum': 995, 'polElectron': -1.0, 'polPositron': 1.0, 'processId': 250127, 'processName': '2f_z_bhabhang', 'run': 250127}
        #print(meta)
        
    n_tot_procpol = meta['nEvtSum']
    
    loc, pol = parse_sample_path(src_file)
    #pol_key = get_pol_key(meta['polElectron'], meta['polPositron'])

    #print(src_file, meta['polElectron'], meta['polPositron'])
    w = weighting(meta['crossSection'], pol, n_tot_procpol)
    #check = w*n_tot_procpol/(meta['crossSection']*1000)/4
    
    print(f'{meta["processName"]} ({pol}) wt:{w:.3f} | {n_tot_procpol} events (branch {branch})')
        
    for presel in ['llHH', 'vvHH', 'qqHH']:
        with ur.open(f'{DATA_ROOT}/{branch}_PreSelection_{presel}.root') as rf:
            passed = rf['eventTree']['preselPassed'].array()
            
            n_items = len(passed)
            n_passed = np.sum(passed)
            
            print(f'  {presel}: {n_items} -> {n_passed}' + (f' ({(n_passed*w):.2f})' if n_passed > 0 else ''))
            
    print('')

2f_z_bhabhang ((-1, -1)) wt:156912.551 | 995 events (branch 0)
  llHH: 995 -> 0
  vvHH: 995 -> 0
  qqHH: 995 -> 0

n1n1qqh ((-1, 1)) wt:0.724 | 995 events (branch 100)
  llHH: 995 -> 0
  vvHH: 995 -> 0
  qqHH: 995 -> 0

n1n1qqh ((1, -1)) wt:0.008 | 995 events (branch 101)
  llHH: 995 -> 0
  vvHH: 995 -> 0
  qqHH: 995 -> 0

n23n23qqh ((-1, 1)) wt:0.611 | 995 events (branch 102)
  llHH: 995 -> 0
  vvHH: 995 -> 0
  qqHH: 995 -> 0

n23n23qqh ((1, -1)) wt:0.016 | 995 events (branch 103)
  llHH: 995 -> 0
  vvHH: 995 -> 0
  qqHH: 995 -> 0

e1e1hh ((-1, -1)) wt:0.000 | 990 events (branch 104)
  llHH: 990 -> 0
  vvHH: 990 -> 0
  qqHH: 990 -> 0

e1e1hh ((-1, 1)) wt:0.027 | 990 events (branch 105)
  llHH: 990 -> 0
  vvHH: 990 -> 0
  qqHH: 990 -> 0

e1e1hh ((1, -1)) wt:0.001 | 990 events (branch 106)
  llHH: 990 -> 0
  vvHH: 990 -> 0
  qqHH: 990 -> 0

e1e1hh ((1, 1)) wt:0.000 | 990 events (branch 107)
  llHH: 990 -> 0
  vvHH: 990 -> 0
  qqHH: 990 -> 0

e1e1qqh ((-1, -1)) wt:0.005 | 990 events (bra

In [60]:
raw = zhh.get_raw_files(debug=True)
raw

['/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250127.P2f_z_bhabhang.eL.pR.n001.d_dstm_10609_0.slcio',
 '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhaNg/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250128.P2f_z_bhabhang.eR.pL.n001.d_dstm_10618_0.slcio',
 '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhag/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250102.P2f_z_bhabhag.eL.pR.n001.d_dstm_10366_0.slcio',
 '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_bhabhag/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250104.P2f_z_bhabhag.eR.pL.n001.d_dstm_10366_0.slcio',
 '/pnfs/desy.de/ilc/prod/ilc/ild/copy/dst-merged/500-TDR_ws/2f_Z_hadronic/ILD_l5_o1_v02/v02-00-01/rv02-00-01.sv02-00-01.mILD_l5_o1_v02.E500-TDR_ws.I250114.P2f_z_h.eL.pR.n001.d_dstm_10410_1